In [26]:
import cv2
import pytesseract
from PIL import Image
import numpy as np

In [27]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [28]:

def capture_image():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        cv2.imshow('Press Space to Capture', frame)
        if cv2.waitKey(1) & 0xFF == ord(' '):
            cv2.imwrite('captured_image.jpg', frame)
            break
    cap.release()
    cv2.destroyAllWindows()

def preprocess_image(image_path):
    # Load the image
    img = cv2.imread(image_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply thresholding
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
    
    # Remove noise
    kernel = np.ones((1, 1), np.uint8)
    processed_img = cv2.dilate(thresh, kernel, iterations=1)
    processed_img = cv2.erode(processed_img, kernel, iterations=1)
    
    # Save the processed image
    processed_image_path = 'processed_image.jpg'
    cv2.imwrite(processed_image_path, processed_img)
    
    return processed_image_path

def extract_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

def save_text_to_file(text, file_path):
    with open(file_path, 'w') as file:
        file.write(text)

capture_image()
image_path = 'captured_image.jpg'
processed_image_path = preprocess_image(image_path)
extracted_text = extract_text_from_image(processed_image_path)
print(extracted_text)

# Save the extracted text to a file
text_file_path = 'extracted_text.txt'
save_text_to_file(extracted_text, text_file_path)

In [29]:
# Import necessary libraries
from ctransformers import AutoModelForCausalLM

def load_llm():
    # Load the locally downloaded model here
    llm = AutoModelForCausalLM.from_pretrained(
        "deepseek-r1:14b",
        model_type="llama",
        config={
            'max_new_tokens': 3000,
            'temperature': 0.01,
            'context_length': 3000
        }
    )
    return llm

# Load the LLaMA model
model = load_llm()

# Read the text file
with open('extracted_text.txt', 'r') as file:
    text_data = file.read()

# Process the text data with the model
result = model.generate(text_data)

# Print the result
print(result)

AttributeError: 'dict' object has no attribute 'config'